In [ ]:
!pip install progressbar2
import matplotlib.pyplot as plt
import numpy as np
import os
%tensorflow_version 1.x
import tensorflow as tf
import time
import progressbar
from keras.datasets import mnist, cifar10

from tqdm import trange

In [16]:
def build_linear_model_two(is_training, learning_rate, reuse, **kwargs):
  with tf.variable_scope('', reuse=reuse):  
    def _linear_with_bias(input_tensor, input_dim, output_dim, layer_name):
      with tf.variable_scope(layer_name):
        with tf.variable_scope("weights"):
          W = tf.get_variable("W", shape=(input_dim, output_dim))

        with tf.variable_scope("biases"):
          b = tf.get_variable("b", shape=(output_dim))

        return tf.matmul(input_tensor, W) + b

    def _linear_with_normalization(input_tensor, input_dim, output_dim, layer_name, eps=1e-9):
      with tf.variable_scope(layer_name):
        with tf.variable_scope("weights"):
          W = tf.get_variable("W", shape=(input_dim, output_dim))

        with tf.variable_scope("gamma"):
          gamma = tf.get_variable("gamma", shape=(1, output_dim))

        with tf.variable_scope("beta"):   
          beta = tf.get_variable("beta", shape=(1, output_dim))
        x = tf.matmul(input_tensor, W)
        mean, var = tf.nn.moments(x, axes=[1], keep_dims=True)
        return (x-mean)/tf.math.sqrt(var + eps) * (gamma+1) + beta
        
    def _normalization(input_tensor, input_dim, layer_name, number, eps = 1e-9):
      with tf.variable_scope(layer_name+"."+str(number)):
        with tf.variable_scope("gamma"):
          gamma = tf.get_variable("gamma", shape=(1, 1, 1, input_dim[3]))

        with tf.variable_scope("beta"):   
          beta = tf.get_variable("beta", shape=(1, 1, 1, input_dim[3]))

        mean, var = tf.nn.moments(input_tensor, axes=[0,1,2], keep_dims=True)
        return (input_tensor-mean)/tf.math.sqrt(var + eps) * (gamma+1) + beta
        
    def _convolution(input_tensor, output_dim, layer_name, number):
      with tf.variable_scope(layer_name+str(number)):
        with tf.variable_scope("conv"):
          filters = tf.get_variable("filters", shape=(output_dim[0], output_dim[1], output_dim[2], output_dim[3]))

      return tf.nn.conv2d(input_tensor, filters, [1, 1, 1, 1], padding="SAME")
      
    def _activation(input_tensor):
      return tf.nn.relu(input_tensor)
    
    def _dropout(input_tensor, dropout, training):
      if training:
        return tf.nn.dropout(input_tensor, 1.0-dropout)
      else:
        return input_tensor
    
    class Model(object):
      pass

    model = Model()

    # Define input
    image_size = kwargs["image_size"]
    images = tf.placeholder(shape=(None, image_size[0], image_size[1], image_size[2]), dtype=tf.float32, name="images")
    labels = tf.placeholder(shape=(None), dtype=tf.int32, name="labels")

    model.inputs = [images, labels]

    tf.summary.image('input_images', images)

    x = images
    mean_b, var_b = tf.nn.moments(x, axes=[1, 2], keep_dims=True)
    x = (x-mean_b)/tf.math.sqrt(var_b + 1e-9)
    sizes = kwargs["sizes"]
    last_size = image_size[2]
    tmp_size = image_size[1]
    for i in range(len(sizes)):
      rx = x
      x = _convolution(x, (3, 3, last_size, sizes[i][0]), "layer"+str(i), 1)
      x = _normalization(x, x.shape, "layer"+str(i), 1)
      x = _activation(x)
      x = _dropout(x, kwargs["dropout"], is_training)      
      
      if(last_size < sizes[i][1]):
        rx = _convolution(rx, (1, 1, last_size, sizes[i][1]), "layer" + str(i), 4)
        last_size = sizes[i][1]
      
      x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
      rx = tf.nn.max_pool(rx, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
      tmp_size = tmp_size/2
      
      x = rx + x


    x = tf.reshape(x, (kwargs["batch_size"], -1))   
    x = _linear_with_normalization(x, input_dim=tmp_size*tmp_size*last_size, output_dim=500, layer_name='linear'+str(len(sizes))) 
    x = _linear_with_normalization(x, input_dim=500, output_dim=100, layer_name='linear'+str(len(sizes)+1)) 
    logits = _linear_with_bias(input_tensor=x, input_dim=100, output_dim=10, layer_name='linear'+str(len(sizes)+2))
                               
    # Compute probabilities
    model.probs = tf.nn.softmax(logits)

    # Compute loss
    model.loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, 
                                                        logits=logits)

    # Optimize
    model.train_op = tf.train.AdamOptimizer().minimize(model.loss)
    return model

In [17]:
def evaluate_linear_model(sess, model, test_images, test_labels, batch_size, **kwargs):
  steps_per_epoch = test_images.shape[0] // batch_size
  
  correct_predictions = 0.0
  total_predictions = 0
  for i in range(steps_per_epoch):
    batch_x = test_images[i*batch_size : (i+1)*batch_size]
    batch_y = test_labels[i*batch_size : (i+1)*batch_size]
    
    images, labels = model.inputs
    model_probs = sess.run(model.probs, feed_dict={
                                images: batch_x,
                                labels: batch_y,
                           })

    test_labels = np.array(test_labels, dtype=np.int32).reshape(test_labels.shape[0])
    correct_predictions += np.sum(np.argmax(model_probs, axis=1) == batch_y)
    total_predictions += batch_size
    
  return correct_predictions / total_predictions

In [18]:
def _create_nn(train_images, train_labels, test_images, test_labels, **params):
  tf.reset_default_graph()

  train_model = build_linear_model_two(is_training=True, reuse=False, **params)
  eval_model = build_linear_model_two(is_training=False, reuse=True, **params)

  
  with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    timestamp = int(time.time())
    writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'linear_model_%d' % timestamp), 
                                   sess.graph,
                                   flush_secs=5)
  

    steps_per_epoch = train_images.shape[0] // params['batch_size'] 

    loss_history = []
    for epoch in range(params['num_epochs']):    
      perm = np.random.permutation(train_images.shape[0])
      train_images = train_images[perm]
      train_labels = train_labels[perm]
      with progressbar.ProgressBar(max_value = steps_per_epoch) as bar:
        for i in range(steps_per_epoch):
          bar.update(i)
          step = epoch * steps_per_epoch + i

          batch_x = train_images[i*params['batch_size'] : (i+1)*params['batch_size']]
          batch_y = train_labels[i*params['batch_size'] : (i+1)*params['batch_size']]
          images, labels = train_model.inputs
          
          _ = sess.run([train_model.train_op], feed_dict={
                images: batch_x,
                labels: batch_y,
          })

        print("Epoch %d: %.4f" % (
            epoch, evaluate_linear_model(sess, eval_model, test_images, test_labels,  **params)))
    
  writer.close()

In [ ]:
params = {
    'batch_size': 64, 
    'num_epochs': 30,
    'learning_rate': 0.01,
    'log_summaries_every': 200,
    'dropout': 0.1,
    'sizes': [(28, 28, 1), (64, 64, 1)],
    'image_size': (28, 28, 1),
    'pixels': 3072

}
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = np.array(train_images, dtype=np.float32) / 255
test_images = np.array(test_images, dtype=np.float32) / 255

train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
print(train_images.shape, test_images.shape)


train_labels = np.array(train_labels, dtype=np.int32).reshape(train_labels.shape[0])
test_labels = np.array(test_labels, dtype=np.int32).reshape(test_labels.shape[0])

_create_nn(train_images, train_labels, test_images, test_labels, **params)